# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [83]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import imageio as im
import datetime
import random as rn
import tensorflow as tf
from keras import backend as K
import scipy
from PIL import Image, ImageFilter, ImageEnhance
from skimage.transform import resize
from skimage.io import imread
from tensorflow.keras.layers import Dropout

We set the random seed so that the results don't vary drastically.

In [84]:
np.random.seed(30)
rn.seed(30)
tf.keras.utils.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [85]:
train_doc = np.random.permutation(open('C:/Users/I350694/Downloads/Project_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('C:/Users/I350694/Downloads/Project_data/Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size
source_path = 'C:/Users/I350694/Downloads/Project_data/Project_data/train'
print(len(train_doc))
img_idx = [1,3,5,7,9,12,13,15,17,19,20,23,25,28,29]

663


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [86]:
def generator(source_path, folder_list, batch_size):
    print('\nSource path = ', source_path, '; batch size =', batch_size)
    #img_idx = seq_idx #create a list of image numbers you want to use for a particular video
    x = len(img_idx)
    y, z = 125, 125
    while True:
        # doubling the data for augmentation
       # if is_train and augmention:
       #     t = np.concatenate((np.random.permutation(folder_list), np.random.permutation(folder_list)))
       # else:
        t = np.random.permutation(folder_list)
            
        if (len(t)%batch_size) == 0:
            num_batches = int(len(t)/batch_size)
        else:
            num_batches = len(t)//batch_size + 1
            
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                #handling remaining datapoints
                folder_idx = folder + (batch*batch_size)
                if folder_idx >= len(t):
                    break
                folder_str = t[folder_idx]
                imgs = os.listdir(source_path+'/'+ folder_str.split(';')[0]) # read all the images in the folder
                # randomly enabling augmentation and augmentation type

                #if is_train and augmention and rn.randint(0,1) == 1:
                #    aug_type = rn.randint(0, 4) #randomly selecting augmentation type
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ folder_str.strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # making the rectangle images into square by cropping sides
                    # so the aspect ration can be mantained while resizing.
                    if image.shape[1] > image.shape[0]:
                        diff_px = image.shape[1] - image.shape[0]
                        crop_start = diff_px//2
                        crop_end = crop_start + image.shape[0]
                        image = image[:, crop_start:crop_end]
                    elif image.shape[0] > image.shape[1]:
                        diff_px = image.shape[0] - image.shape[1]
                        crop_start = diff_px//2
                        crop_end = crop_start + image.shape[1]
                        image = image[:, crop_start:crop_end]

                   # resized_im = imresize(image, 125,125)
                    import cv2
                    size = (125,125)
                    resized_im = cv2.resize(image, size)
                    
                    batch_data[folder,idx,:,:,0] = resized_im[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = resized_im[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = resized_im[:,:,2]/255 #normalise and feed in the image

                batch_labels[folder, int(folder_str.strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
                

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [87]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:/Users/I350694/Downloads/Project_data/Project_data/train'
val_path = 'C:/Users/I350694/Downloads/Project_data/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15 # choose the number of epochs
print ('# epochs =', num_epochs)
img_idx = [1,3,5,7,9,12,13,15,17,19,20,23,25,28,29]
print(len(img_idx))

# training sequences = 663
# validation sequences = 100
# epochs = 15
15


## First Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [65]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
input_shape = (len(img_idx), 125, 125, 3)
model = Sequential()
# First Convolutional Layer
model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=2))

# Second Convolutional Layer
model.add(Conv3D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=2))

# Flattening
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(units=128, activation='relu'))
Dropout(0.5),
model.add(Dense(units=5, activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [66]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_4 (Conv3D)                    │ (None, 13, 123, 123, 32)    │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_4 (MaxPooling3D)       │ (None, 6, 61, 61, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_5 (Conv3D)                    │ (None, 4, 59, 59, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_5 (MaxPooling3D)       │ (None, 2, 29, 29, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 107648)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 128)                 │      13,779,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,837,701 (52.79 MB)

 Trainable params: 13,837,701 (52.79 MB)

 Non-trainable params: 0 (0.00 B)

None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [67]:
print(train_path)

C:/Users/I350694/Downloads/Project_data/Project_data/train


In [68]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [78]:
pip show keras

Name: keras
Version: 3.1.1
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: c:\users\i350694\appdata\local\anaconda3\lib\site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, rich
Required-by: tensorflow-intel


In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
#filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath = model_name + 'model.keras'
#filepath += '.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [71]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [47]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None,  initial_epoch=0)
#workers=1,


Source path =  C:/Users/I350694/Downloads/Project_data/Project_data/train ; batch size = 32
Epoch 1/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - categorical_accuracy: 0.1778 - loss: 4.3441
Source path =  C:/Users/I350694/Downloads/Project_data/Project_data/val ; batch size = 32

Epoch 1: saving model to model_init_2024-04-0318_21_05.873025/model.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 103s 5s/step - categorical_accuracy: 0.1786 - loss: 4.2701 - val_categorical_accuracy: 0.1797 - val_loss: 1.2029 - learning_rate: 0.0010
Epoch 2/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.3349 - loss: 1.4807
Epoch 2: saving model to model_init_2024-04-0318_21_05.873025/model.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 78s 4s/step - categorical_accuracy: 0.3367 - loss: 1.4777 - val_categorical_accuracy: 0.3359 - val_loss: 0.9785 - learning_rate: 0.0010
Epoch 3/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - categorical_accuracy: 0.5415 - loss: 1.1669
Epoch 3: saving model to model_init_2024-04-0318_21_05.8

##### Training Accuracy is 84% and Validation accuracy is 62%. when image size was 150, 150. The model showing that its overfitted
##### Training Accuracy is 79% and Validation accuracy is 67%. when image size was 125, 125. The model showing that its overfitted
### Second Model


In [846]:
model = Sequential()

model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv3D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling3D(pool_size=2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

In [847]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_76"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_214 (Conv3D)                  │ (None, 13, 148, 148, 32)    │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_215 (Conv3D)                  │ (None, 11, 146, 146, 64)    │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_207 (MaxPooling3D)     │ (None, 5, 73, 73, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 5, 73, 73, 64)       │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_216 (Conv3D)                  │ (None, 3, 71, 71, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_208 (MaxPooling3D)     │ (None, 1, 35, 35, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 1, 35, 35, 128)      │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_68 (Flatten)                 │ (None, 156800)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_134 (Dense)                    │ (None, 512)                 │      80,282,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_135 (Dense)                    │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 80,566,789 (307.34 MB)

 Trainable params: 80,565,381 (307.33 MB)

 Non-trainable params: 1,408 (5.50 KB)

None


In [849]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None,  initial_epoch=0)


Source path =  C:/Users/I350694/Downloads/Project_data/Project_data/train ; batch size = 32
Epoch 1/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - categorical_accuracy: 0.4497 - loss: 2.6746 
Source path =  C:/Users/I350694/Downloads/Project_data/Project_data/val ; batch size = 32

Epoch 1: saving model to model_init_2024-04-0300_30_18.234938/model.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 385s 16s/step - categorical_accuracy: 0.4525 - loss: 2.6617 - val_categorical_accuracy: 0.2500 - val_loss: 5.9680 - learning_rate: 0.0010
Epoch 2/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - categorical_accuracy: 0.7692 - loss: 0.6637 
Epoch 2: saving model to model_init_2024-04-0300_30_18.234938/model.keras
21/21 ━━━━━━━━━━━━━━━━━━━━ 298s 14s/step - categorical_accuracy: 0.7690 - loss: 0.6620 - val_categorical_accuracy: 0.2500 - val_loss: 2.1867 - learning_rate: 0.0010
Epoch 3/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - categorical_accuracy: 0.8763 - loss: 0.3490 
Epoch 3: saving model to model_init_2024-04-030

###### Increasing the layers, we could achieve the Training Accuracry to 98% but Validation accuracy dropped to 35%. The model is overfitted
### Third Model

In [48]:
print(input_shape)

(15, 125, 125, 3)


In [49]:
from keras.layers import Input,MaxPooling2D, TimeDistributed, Conv2D, Bidirectional, BatchNormalization,GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
model = Sequential()
model.add(Input(input_shape))
model.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())
Dropout(0.2),

model.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())
Dropout(0.2),

model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(TimeDistributed(Dense(516, activation='relu')))
model.add(BatchNormalization())
Dropout(0.5),

model.add(GRU(128))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax'))

In [50]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_18                  │ (None, 15, 123, 123, 32)    │             896 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_19                  │ (None, 15, 61, 61, 32)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 15, 61, 61, 32)      │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_20                  │ (None, 15, 59, 59, 64)      │          18,496 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_21                  │ (None, 15, 29, 29, 64)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 15, 29, 29, 64)      │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_22                  │ (None, 15, 64)              │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_23                  │ (None, 15, 516)             │          33,540 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 15, 516)             │           2,064 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 128)                 │         248,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 304,601 (1.16 MB)

 Trainable params: 303,121 (1.16 MB)

 Non-trainable params: 1,480 (5.78 KB)

None


In [51]:
print('steps_per_epoch :', steps_per_epoch , 'num_epochs', num_epochs, 'validation_steps: ', validation_steps)

steps_per_epoch : 21 num_epochs 15 validation_steps:  4


In [52]:
#Got Memory out error/ResourceExhaustedError hence, reducing the batchsize to 15, epcochs to 10
batch_size = 10
num_epochs = 10
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [53]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None,  initial_epoch=0)


Source path =  C:/Users/I350694/Downloads/Project_data/Project_data/train ; batch size = 10
Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.3966 - loss: 1.7718
Source path =  C:/Users/I350694/Downloads/Project_data/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-04-0318_21_05.873025/model.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 205s 3s/step - categorical_accuracy: 0.3973 - loss: 1.7678 - val_categorical_accuracy: 0.3700 - val_loss: 1.5899 - learning_rate: 0.0010
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.5806 - loss: 1.0559
Epoch 2: saving model to model_init_2024-04-0318_21_05.873025/model.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 185s 3s/step - categorical_accuracy: 0.5805 - loss: 1.0563 - val_categorical_accuracy: 0.2400 - val_loss: 1.6220 - learning_rate: 0.0010
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - categorical_accuracy: 0.5972 - loss: 0.9977
Epoch 3: saving model to model_init_2024-04-0318_21_05.

##### Training Accuracy is 73% and Validation accuracy is 53% for when image size was 150, 150. The model showing that its overfitted
##### Training Accuracy is 73% and Validation accuracy is 59% for when image size was 125, 125. The model showing that its overfitted

## Fourth Model

In [54]:
from keras.layers import Input,MaxPooling2D, TimeDistributed, Conv2D, Bidirectional, BatchNormalization,GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
model = Sequential()
model.add(Input(input_shape))
model.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)
model.add(BatchNormalization())
Dropout(0.2),

model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(TimeDistributed(Dense(516, activation='relu')))
model.add(BatchNormalization())
Dropout(0.2),

model.add(GRU(256))
model.add(BatchNormalization())
Dropout(0.5)
model.add(Dense(5, activation='softmax'))

In [56]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_24                  │ (None, 15, 123, 123, 32)    │             896 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_25                  │ (None, 15, 61, 61, 32)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 15, 61, 61, 32)      │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_26                  │ (None, 15, 32)              │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_27                  │ (None, 15, 516)             │          17,028 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 15, 516)             │           2,064 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 256)                 │         594,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 616,857 (2.35 MB)

 Trainable params: 615,249 (2.35 MB)

 Non-trainable params: 1,608 (6.28 KB)

None


In [57]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None,  initial_epoch=0)

Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.3633 - loss: 1.8955
Epoch 1: saving model to model_init_2024-04-0318_21_05.873025/model.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - categorical_accuracy: 0.3637 - loss: 1.8918 - val_categorical_accuracy: 0.1800 - val_loss: 1.5950 - learning_rate: 0.0010
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.4863 - loss: 1.2297
Epoch 2: saving model to model_init_2024-04-0318_21_05.873025/model.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - categorical_accuracy: 0.4861 - loss: 1.2296 - val_categorical_accuracy: 0.3600 - val_loss: 1.5589 - learning_rate: 0.0010
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.4876 - loss: 1.1853
Epoch 3: saving model to model_init_2024-04-0318_21_05.873025/model.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 129s 2s/step - categorical_accuracy: 0.4875 - loss: 1.1857 - val_categorical_accuracy: 0.2100 - val_loss: 1.5607 - learning_rate: 0.0010
E

##### Training Accuracy is 63% and Validation accuracy is 35% for when image size was 150, 150. The model showing that its overfitted
##### Training Accuracy is 64% and Validation accuracy is 44% for when image size was 125, 125. The model showing that its overfitted
##### Reducing the number of nodes or making the network shallow should resolve overfitting issues
## Fifth Model


In [89]:
batch_size = 10
num_epochs = 10
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [90]:
from keras.layers import Input,MaxPooling2D, TimeDistributed, Conv2D, Bidirectional, BatchNormalization,GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
model = Sequential()
model.add(Input(input_shape))
model.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'))
)
model.add(TimeDistributed(
    MaxPooling2D((2,2)))
)

model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(TimeDistributed(Dense(64, activation='relu')))
model.add(BatchNormalization())


model.add(GRU(32))
model.add(Dense(5, activation='softmax'))

In [91]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_36                  │ (None, 15, 123, 123, 32)    │             896 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_37                  │ (None, 15, 61, 61, 32)      │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_38                  │ (None, 15, 32)              │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_39                  │ (None, 15, 64)              │           2,112 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_28               │ (None, 15, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_6 (GRU)                          │ (None, 32)                  │           9,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,837 (50.14 KB)

 Trainable params: 12,709 (49.64 KB)

 Non-trainable params: 128 (512.00 B)

None


In [92]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None,  initial_epoch=0)


Source path =  C:/Users/I350694/Downloads/Project_data/Project_data/train ; batch size = 10
Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.3208 - loss: 1.5036
Source path =  C:/Users/I350694/Downloads/Project_data/Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2024-04-0321_56_31.493767/model-00001-1.45882-0.32985-1.58573-0.29000.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 160s 2s/step - categorical_accuracy: 0.3209 - loss: 1.5030 - val_categorical_accuracy: 0.2900 - val_loss: 1.5857 - learning_rate: 0.0010
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0.4208 - loss: 1.3244
Epoch 2: saving model to model_init_2024-04-0321_56_31.493767/model-00002-1.30335-0.42537-1.54786-0.39000.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - categorical_accuracy: 0.4209 - loss: 1.3241 - val_categorical_accuracy: 0.3900 - val_loss: 1.5479 - learning_rate: 0.0010
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - categorical_accuracy: 0

#### Training accuracy is 57% and validation 32%. However this model does not seems to be a good model